# sanity check Livre

In [ ]:
from mongo_livre import Livre

In [ ]:
for livre in Livre.get_entries():
    print(livre)


        Titre: Germinal
        Auteur: Émile Zola
        Editeur: Le Livre de Poche
        

        Titre: Le Rouge et le Noir
        Auteur: Stendhal
        Editeur: None
        


# add_book

In [ ]:
from typing import Optional, Dict
from mongo_auteur import Auteur, AuthorChecker
from mongo_episode import Episode
from mongo_livre import Livre
from mongo import Editeur
from datetime import datetime


def add_book(
    episode_date: str,
    title: str,
    author: str,
    editor: Optional[str] = None,
    verbose: bool = False,
) -> Livre:
    """Adds a book to the database and returns its details.

    Adds a book to the database while ensuring that a valid author is provided and that the editor is optional.
    The episode corresponding to the given date is verified. If the episode does not exist, or if the author is missing or improperly formatted,
    a ValueError is raised.

    Args:
        episode_date (str): The date of the associated episode. Must follow the format '%d/%m/%Y'.
        title (str): The title of the book.
        author (str): The author's name.
        editor (Optional[str], optional): The editor's name. Defaults to None.
        verbose (bool, optional): If True, prints details of the author correction. Defaults to False.

    Returns:
        Livre: An instance of Livre representing the added book.

    Raises:
        ValueError: If the author parameter is empty.
        ValueError: If there is no episode matching the provided date.
    """
    if not author:
        raise ValueError("Author parameter must not be empty.")

    episode = Episode.from_date(datetime.strptime(episode_date, "%d/%m/%Y"))
    if not episode:
        raise ValueError(
            f"Episode with date {episode_date} does not exist in the database."
        )
    editeur_inst = Editeur(editor) if editor else None

    ac = AuthorChecker(episode)

    auteur_corrige_dict = ac.check_author(author, return_details=False, verbose=verbose)
    if not auteur_corrige_dict:
        raise ValueError(f"Corrected Author empty starting from Author {author}.")
    auteur_inst = Auteur(auteur_corrige_dict)
    auteur_inst.keep()
    livre = Livre.with_details(titre=title, auteur=auteur_inst, editeur=editeur_inst)
    livre.keep()

    return livre

In [ ]:
from mongo_episode import Episodes

episodes = Episodes()
episodes.get_entries(limit=2)
last_episode = episodes[0]
last_episode

_oid: 679649e267b093aaae847524
Date: 26 Jan 2025 10:59
Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/f421bbf9-5e6e-4411-85a6-7c6a318d2073/14007-26

In [ ]:
titre = "Le Seigneur des Anneaux"
auteur = "J.R. Tolkien"
date_episode = "26/01/2025"

add_book(date_episode, title=titre, author=auteur, verbose=True)

Trouvé avec mongodb:auteurs: J.R.R Tolkien


Le Seigneur des Anneaux

In [ ]:
titre = "Un titre et auteur farfeulu"
auteur = "Guillaume Ramelet-Caffeau"
date_episode = "26/01/2025"

add_book(date_episode, title=titre, author=auteur, verbose=True)

Score insuffisant 0 avec web search: La requête Google ne fournit aucune information indiquant que Guillaume Ramelet-Caffeau est un auteur de livres. Les résultats sont principalement liés à des arbres généalogiques et des archives, sans mention de publications littéraires ou d'activités d'écriture. Par conséquent, il est peu probable que Guillaume Ramelet-Caffeau soit un auteur de livres.


ValueError: Corrected Author empty starting from Author Guillaume Ramelet-Caffeau.